In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


# 1) Data Configuration

## configuring training data

In [2]:
train_data = pd.read_csv("/kaggle/input/playground-series-s4e11/train.csv")
# first imma get rid of the columns that obviously have no relation to whether one has depression or not
# removing columns 'id' and 'Name'
train_data.drop(['id', 'Name'], axis=1, inplace=True)

# function to check for NaN values in a column
def checknan(colname):
    print(train_data[train_data[colname].isna()])

# function to fillna with either mean, median, 0, or another column
def fillNaN(colnamelist, type, othercol=None):
    if type == 'mean':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[colname].mean()}, inplace = True)
    elif type == 'median':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[colname].median()}, inplace = True)
    elif type == 0:
        for colname in colnamelist:
            train_data.fillna({colname: 0}, inplace = True)
    elif type == 'othercol':
        for colname in colnamelist:
            train_data.fillna({colname: train_data[othercol]}, inplace = True)
    print('fillNaN complete')
        
# dealing with NaNs in column: Profession
train_data['Profession'] = train_data['Profession'].fillna(train_data['Working Professional or Student'])

# dealing with NaNs in columns: Academic Pressure & Work Pressure
train_data['Work Pressure'] = train_data['Work Pressure'].fillna(train_data['Academic Pressure'])
train_data['Pressure'] = train_data['Work Pressure']
train_data.drop(['Work Pressure', 'Academic Pressure'], axis=1, inplace=True)
train_data.fillna({'Pressure': 0}, inplace=True)

# dealing with NaNs in column: CGPA
print(train_data['CGPA'].median())
train_data['CGPA'] = train_data['CGPA'].fillna(train_data['CGPA'].median())

# dealing with NaNs in columns: Study Satisfaction and Job Satisfaction
train_data['Job Satisfaction'] = train_data['Job Satisfaction'].fillna(train_data['Study Satisfaction'])
train_data.fillna({'Job Satisfaction': train_data['Job Satisfaction'].mean()}, inplace=True)
train_data['Career Satisfaction'] = train_data['Job Satisfaction']
train_data.drop(['Job Satisfaction', 'Study Satisfaction'], axis=1, inplace=True)

# Dietary Habits and Degree columns
train_data.fillna({'Degree': 'Unknown Degree', 'Dietary Habits': 'Moderate'}, inplace=True)

# Financial Stress
fillNaN(['Financial Stress'], 'mean')

traincities = train_data['City'].unique()


7.77
fillNaN complete


## configuring test data

In [3]:
test_data = pd.read_csv("/kaggle/input/playground-series-s4e11/test.csv")
# can only get rid of the 'Name' column as the id column is needed for submission
test_data.drop(['Name'], axis=1, inplace=True)

# function to check for NaN values in a column
def checknan(colname):
    print(test_data[test_data[colname].isna()])

# function to fillna with either mean, median, 0, or another column
def fillNaN(colnamelist, type, othercol=None):
    if type == 'mean':
        for colname in colnamelist:
            test_data.fillna({colname: train_data[colname].mean()}, inplace = True) # fillna with training data mean
    elif type == 'median':
        for colname in colnamelist:
            test_data.fillna({colname: train_data[colname].median()}, inplace = True) # fillna with training data median
    elif type == 0:
        for colname in colnamelist:
            test_data.fillna({colname: 0}, inplace = True)
    elif type == 'othercol':
        for colname in colnamelist:
            test_data.fillna({colname: test_data[othercol]}, inplace = True)
    print('fillNaN complete')
        
# dealing with NaNs in column: Profession
test_data['Profession'] = test_data['Profession'].fillna(test_data['Working Professional or Student'])

# dealing with NaNs in columns: Academic Pressure & Work Pressure
test_data['Work Pressure'] = test_data['Work Pressure'].fillna(test_data['Academic Pressure'])
test_data['Pressure'] = test_data['Work Pressure']
test_data.drop(['Work Pressure', 'Academic Pressure'], axis=1, inplace=True)
test_data.fillna({'Pressure': 0}, inplace=True)

# dealing with NaNs in column: CGPA
print(test_data['CGPA'].median())
test_data['CGPA'] = test_data['CGPA'].fillna(train_data['CGPA'].median())

# dealing with NaNs in columns: Study Satisfaction and Job Satisfaction
test_data['Job Satisfaction'] = test_data['Job Satisfaction'].fillna(test_data['Study Satisfaction'])
test_data.fillna({'Job Satisfaction': train_data['Career Satisfaction'].mean()}, inplace=True)
test_data['Career Satisfaction'] = test_data['Job Satisfaction']
test_data.drop(['Job Satisfaction', 'Study Satisfaction'], axis=1, inplace=True)

# Dietary Habits and Degree columns
test_data.fillna({'Degree': 'Unknown Degree', 'Dietary Habits': 'Moderate'}, inplace=True)

# Financial Stress
fillNaN(['Financial Stress'], 'mean')
testcities = test_data['City'].unique()



7.8
fillNaN complete


## manually getting dummy columns for the unique city names -->

In [4]:
# checking if the cities in the train data and test data are the same
print(set(testcities))
print(len(testcities)) #68 cities in the testing data

print(set(traincities))
print(len(traincities)) #98 cities in the training data

# creating a list of all cities 
allcities = list(set(testcities) | set(traincities)) # length of all cities = 124
print(allcities)
# manually getting dummy city columns for both training and test datasets
for name in allcities:
    for df in [train_data, test_data]:
        df['name'] = False

# manually inputting the city for each row
# (if the person in that row resides in said city, value of the city column becomes from False to True)
for df in [train_data, test_data]:
    for index, row in df.iterrows():
        cityname = row['City']
        df.at[index, cityname] = True
# removing the city column, its not needed anymore yay
for df in [train_data, test_data]:
    df.drop('City', axis=1 , inplace=True)

{'Leela', 'Ira', 'Is Kanpur', 'Vaikot', 'Thani', 'Avni', 'Saurav', 'Vaishnavi', 'Patna', 'Ghopal', 'Nalini', 'No', 'Jaipur', 'More Delhi', 'Kanpur', 'Golkata', 'Chennai', 'Parth', 'City', 'Mhopal', 'Less Delhi', 'Vadodara', 'Meerut', 'Indore', 'Kalyan', 'Varanasi', 'Vasai-Virar', 'Bhopal', 'Ghaziabad', 'Ludhiana', 'Less than 5 hours', 'Malyan', 'Bhavna', 'Delhi', 'Rajkot', 'Keshav', 'Pratyush', 'Chemist', 'San Vasai-Virar', 'Saanvi', 'Hrithik', 'Ahmedabad', 'Unaly', 'Surat', 'Agra', 'Vikram', 'Pune', 'Srinagar', 'Aditi', 'Bangalore', 'Lucknow', 'Vidhi', 'Rolkata', 'Siddhesh', 'Thane', 'Nashik', 'Faridabad', 'Sara', 'Nagpur', 'Mumbai', 'Vidya', 'Visakhapatnam', 'Pratham', 'Lawyer', 'Abhinav', 'Kolkata', 'Hyderabad', 'No.12'}
68
{'Aditya', 'Chhavi', 'Anvi', 'MCA', 'Chennai', 'Vadodara', 'Bhopal', 'Ghaziabad', 'Less than 5 Kalyan', 'Mira', 'Kashish', 'Galesabad', 'Krishna', 'Krinda', 'Jhanvi', 'Kolkata', 'Khushi', 'Pooja', 'Patna', '3.0', 'M.Tech', 'Harsha', 'Ishkarsh', 'City', 'Less Delh

/tmp/ipykernel_17/2583383832.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, cityname] = True
/tmp/ipykernel_17/2583383832.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, cityname] = True
/tmp/ipykernel_17/2583383832.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy(